In [1]:
import cobra

In [23]:
import sys
sys.path.insert(0, '../cue_utils')
from utils import (
                   atomExchangeMetabolite,
                   calculate_cue,
                   calculate_gge,
                   extract_c_fates)

In [4]:
# Load in model file
model = cobra.io.read_sbml_model("../../../GEM-repos/mit1002-core-model/core_314275.5_GP.SBML/core_314275.5_GP.xml")

'' is not a valid SBML 'SId'.


In [10]:
model

Name,
Memory address,131396e90
Number of metabolites,141
Number of reactions,155
Number of genes,317
Number of groups,0
Objective expression,1.0*bio2_biomass - 1.0*bio2_biomass_reverse_58445
Compartments,"e0, c0"


In [6]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd11416_c0,DM_cpd11416_c0,-1000,0,0.00%


In [14]:
cobra.util.solver.linear_reaction_coefficients(model)

{<Reaction bio2_biomass at 0x14f243760>: 1.0}

In [15]:
model.objective = "bio1_biomass"

In [17]:
model.reactions.get_by_id("bio1_biomass").upper_bound

1000.0

In [16]:
cobra.util.solver.linear_reaction_coefficients(model)

{<Reaction bio1_biomass at 0x14f2be9e0>: 1.0}

In [19]:
sol = model.optimize()

In [20]:
sol.objective_value

126.65279127266031

In [21]:
sol

,fluxes,reduced_costs
rxn14414_c0,7.346948e-14,0.000000e+00
rxn14412_c0,-1.000000e+03,-0.000000e+00
rxn14416_c0,-1.000000e+03,-8.296034e-03
rxn14418_c0,0.000000e+00,1.090503e-18
rxn14419_c0,1.000000e+03,-1.870145e-17
...,...,...
EX_cpd00027_e0,-1.000000e+03,0.000000e+00
EX_cpd00211_e0,0.000000e+00,0.000000e+00
EX_cpd00363_e0,-7.163085e+02,0.000000e+00
EX_cpd00001_e0,1.000000e+03,2.986572e-02


In [28]:
c_ex_rxns = atomExchangeMetabolite(model, ex_nomenclature={'e0'})

In [29]:
c_ex_rxns

{'EX_cpd00363_e0': 2,
 'EX_cpd00029_e0': 2,
 'EX_cpd01947_e0': 4,
 'EX_cpd00027_e0': 6,
 'EX_cpd00211_e0': 4,
 'EX_cpd00141_e0': 3,
 'EX_cpd00011_e0': 1,
 'EX_cpd00100_e0': 3,
 'EX_cpd00159_e0': 3}

In [37]:
[model.reactions.get_by_id(r).name for r in c_ex_rxns.keys()]

['Exchange for Ethanol_e0',
 'Exchange for Acetate_e0',
 'Exchange for BDOH_e0',
 'Exchange for D-Glucose_e0',
 'Exchange for Butyrate_e0',
 'Exchange for Propionate_e0',
 'Exchange for CO2_e0',
 'Exchange for Glycerol_e0',
 'Exchange for L-Lactate_e0']

In [38]:
model.reactions.EX_cpd00011_e0

Reaction identifier,EX_cpd00011_e0
Name,Exchange for CO2_e0
Memory address,0x14f2bfc40
Stoichiometry,cpd00011_e0 <=> CO2_e0 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [45]:
co2_rxn = 'EX_cpd00011_e0'

In [39]:
uptake_rxns = sol.fluxes[c_ex_rxns.keys()].pipe(lambda x: x[x<0]).index

In [40]:
uptake_rxns

Index(['EX_cpd00363_e0', 'EX_cpd00027_e0', 'EX_cpd00011_e0'], dtype='object')

In [42]:
uptake = sum([sol.get_primal_by_id(r) * c_ex_rxns[r] for r in uptake_rxns if r != co2_rxn])

In [43]:
uptake

-8168.296892725866

In [46]:
cue = 1 - abs(sol.get_primal_by_id(co2_rxn) / uptake)

In [47]:
cue

0.9099347345475183